In [22]:
%reload_ext autoreload
%autoreload 2

ERROR! Session/line number was not unique in database. History logging moved to new session 1909


In [2]:
import h5py
import pandas as pd
import pickle
from tqdm import tqdm
from li.utils import pairwise_cosine
import time
import logging
import numpy as np
import os
from scipy import sparse


In [3]:
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s][%(levelname)-5.5s][%(name)-.20s] %(message)s'
)
LOG = logging.getLogger(__name__)

def increase_max_recursion_limit():
    """ Increases the maximum recursion limit.
    Source: https://stackoverflow.com/a/16248113
    """
    import sys
    import resource
    resource.setrlimit(resource.RLIMIT_STACK, (2**29, -1))
    sys.setrecursionlimit(10**6)


In [4]:
size = '100K'

LOG.info(f'Loading pca32 data')
data_path = f'data/pca32v2/{size}/dataset.h5'
f = h5py.File(data_path, 'r')
loaded_data = f['pca32'][:, :]
data = pd.DataFrame(loaded_data)
data.index += 1

LOG.info(f'Loading queries')
base_path = f'data/pca32v2/{size}/'
queries_path = f'{base_path}/query.h5'
f2 = h5py.File(queries_path, 'r')
#loaded_queries = f2['emb'][:, :]
loaded_queries = f2['pca32'][:, :]

base_path = f'data/clip768v2/{size}/'
queries_path = f'{base_path}/query.h5'
f2 = h5py.File(queries_path, 'r')
#loaded_queries = f2['emb'][:, :]
loaded_queries_seq = f2['emb'][:, :]

LOG.info(f'Loading clip data')
data_path = f'data/clip768v2/{size}/dataset.h5'
f = h5py.File(data_path, 'r')
loaded_clip_data = f['emb'][:, :]
loaded_clip_data = pd.DataFrame(loaded_clip_data)
loaded_clip_data.index += 1

LOG.info(f'Loading GT')
gt_path = f'data/groundtruth-{size}.h5'
f3 = h5py.File(gt_path, 'r')
loaded_gt = f3['knns'][:, :]


[2023-07-05 16:18:44,109][INFO ][__main__] Loading pca32 data
[2023-07-05 16:18:44,256][INFO ][__main__] Loading queries
[2023-07-05 16:18:44,569][INFO ][__main__] Loading clip data
[2023-07-05 16:18:45,977][INFO ][__main__] Loading GT


In [5]:
from li.BaseLMI import cluster_kmeans_faiss
from sklearn import preprocessing

[2023-07-05 16:18:46,481][INFO ][faiss.loader] Loading faiss with AVX2 support.
[2023-07-05 16:18:46,485][INFO ][faiss.loader] Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'",)
[2023-07-05 16:18:46,488][INFO ][faiss.loader] Loading faiss.
[2023-07-05 16:18:46,526][INFO ][faiss.loader] Successfully loaded faiss.


In [6]:
%time data_prep = preprocessing.normalize(data.values)

CPU times: user 11.6 ms, sys: 7.87 ms, total: 19.5 ms
Wall time: 19.5 ms


In [8]:
%time kmeans_wo, result_wo = cluster_kmeans_faiss(data, n_clusters=10)

CPU times: user 45.4 ms, sys: 12.5 ms, total: 57.9 ms
Wall time: 58 ms


In [9]:
def get_partitioning_quality(kmeans, queries, data, loaded_gt, basic_clustering):
    
    res = kmeans.index.search(np.array(queries).astype(np.float32), 1000)
    
    n_cats_covered = []
    n_objects_covered = []

    for i in tqdm(range(10_000), position=0, leave=True):
        overall_sum = 0
        overall_objects_sum = 0
        argsorted = res[1][i]#np.argsort(res[0][i])[::-1]
        idx = 0
        while overall_sum < 9:
            overall_sum += np.sum(data.loc[loaded_gt[i][:10]].category == argsorted[idx])
            overall_objects_sum += np.sum(basic_clustering == argsorted[idx])
            #overall_objects_sum += np.sum(pred_positions == argsorted[idx])
            idx += 1
        n_cats_covered.append(idx)
        n_objects_covered.append(overall_objects_sum)

    mean_cats_covered = np.mean(np.array(n_cats_covered))
    mean_objects_covered = np.mean(np.array(n_objects_covered))
    LOG.info(f'mean_cats_covered={mean_cats_covered}, mean_objects_covered={mean_objects_covered}')
    return mean_cats_covered, mean_objects_covered, np.array(n_cats_covered), np.array(n_objects_covered)


In [10]:
data['category'] = result_wo

In [11]:
%time mean_cats_covered, mean_objects_covered, cats_all, objs_all = get_partitioning_quality(kmeans_wo, loaded_queries, data, loaded_gt, result_wo)

  0%|          | 0/10000 [00:00<?, ?it/s][2023-07-05 16:18:54,892][INFO ][numexpr.utils] Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2023-07-05 16:18:54,895][INFO ][numexpr.utils] Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
100%|██████████| 10000/10000 [00:20<00:00, 483.53it/s]
[2023-07-05 16:19:15,543][INFO ][__main__] mean_cats_covered=1.9646, mean_objects_covered=22277.1215


CPU times: user 19.7 s, sys: 177 ms, total: 19.9 s
Wall time: 20.8 s


In [12]:
import torch
import torch.utils.data

class LIDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset_x = dataset.drop('category', axis=1).values
        self.dataset_y = dataset.category.values

    def __len__(self):
        return self.dataset_x.shape[0]
    
    def __getitem__(self, idx):
        return self.dataset_x[idx-1], self.dataset_y[idx-1]
        
dataset = LIDataset(data)
        
train_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    sampler=torch.utils.data.SubsetRandomSampler(data.index.values.tolist())
)

In [13]:
from li.model import NeuralNetwork, data_X_to_torch

In [14]:
nn = NeuralNetwork(
    input_dim=data.drop('category', axis=1).shape[1], output_dim=10, lr=0.1, model_type='MLP'
)

In [15]:
losses = nn.train_batch(train_loader, epochs=100, logger=LOG)

[2023-07-05 16:19:15,729][INFO ][__main__] Epochs: 100, step: 10
[2023-07-05 16:19:28,131][INFO ][__main__] Epoch 10 | Loss 0.31040
[2023-07-05 16:19:39,464][INFO ][__main__] Epoch 20 | Loss 0.28142
[2023-07-05 16:19:50,628][INFO ][__main__] Epoch 30 | Loss 0.26895
[2023-07-05 16:20:01,603][INFO ][__main__] Epoch 40 | Loss 0.15553
[2023-07-05 16:20:12,982][INFO ][__main__] Epoch 50 | Loss 0.18940
[2023-07-05 16:20:24,090][INFO ][__main__] Epoch 60 | Loss 0.14437
[2023-07-05 16:20:35,218][INFO ][__main__] Epoch 70 | Loss 0.24336
[2023-07-05 16:20:46,437][INFO ][__main__] Epoch 80 | Loss 0.20678
[2023-07-05 16:20:57,628][INFO ][__main__] Epoch 90 | Loss 0.20918


In [71]:
def get_partitioning_quality_nn(nn, queries, data, loaded_gt, basic_clustering):
    
    res = nn.predict_proba(data_X_to_torch(queries))
    
    n_cats_covered = []
    n_objects_covered = []

    for i in tqdm(range(10_000), position=0, leave=True):
        overall_sum = 0
        overall_objects_sum = 0
        argsorted = np.argsort(res[0][i])[::-1]
        idx = 0
        while overall_sum < 9:
            overall_sum += np.sum(data.loc[loaded_gt[i][:10]].category_nn == argsorted[idx])
            overall_objects_sum += np.sum(basic_clustering == argsorted[idx])
            #overall_objects_sum += np.sum(pred_positions == argsorted[idx])
            idx += 1
        n_cats_covered.append(idx)
        n_objects_covered.append(overall_objects_sum)

    mean_cats_covered = np.mean(np.array(n_cats_covered))
    mean_objects_covered = np.mean(np.array(n_objects_covered))
    LOG.info(f'mean_cats_covered={mean_cats_covered}, mean_objects_covered={mean_objects_covered}')
    return mean_cats_covered, mean_objects_covered, np.array(n_cats_covered), np.array(n_objects_covered)


In [72]:
%%time
category_nn = nn.predict(data_X_to_torch(data.drop(['category', 'category_nn'], axis=1, errors='ignore')))
data['category_nn'] = category_nn
mean_cats_covered, mean_objects_covered, cats_all, objs_all = get_partitioning_quality_nn(
    nn, loaded_queries, data, loaded_gt, category_nn
)
mean_cats_covered, mean_objects_covered

100%|██████████| 2/2 [00:00<00:00, 457.54it/s]
[2023-07-05 16:42:29,186][INFO ][__main__] mean_cats_covered=1.0, mean_objects_covered=12756.5


[0 9 8 7 2 1 4 3 5 6]
[1 5 9 0 2 8 4 7 3 6]
CPU times: user 19.5 ms, sys: 174 µs, total: 19.7 ms
Wall time: 24.4 ms


(1.0, 12756.5)

In [66]:
cats_all, objs_all

(array([1, 1]), array([16287,  9226]))

In [52]:
objs_all

array([16287,  9226,  8676, ..., 11848, 45179, 16287])

In [51]:
cats_all

array([1, 1, 1, ..., 1, 4, 1])

## How to replicate this into the search procedure?

In [26]:
epochs=100
n_categories=10
lr=0.1
model_type='MLP'
batch_size=256

In [27]:
from li.BulkLMI import BulkLMI

In [28]:
blmi = BulkLMI()
blmi.insert(data.drop(['category', 'category_nn'], axis=1, errors='ignore'))
LOG.info('Running deepen (root)')
info_df = pd.DataFrame([], columns=['op', 'time-taken', 'size', '#-objects'])
info_df = blmi.deepen(
    blmi.nodes[(0, )],
    n_categories,
    epochs=epochs,
    lr=lr,
    model_type=model_type,
    batch_size=batch_size,
    info_df=info_df
)

[2023-07-05 16:25:27,903][INFO ][__main__] Running deepen (root)
[2023-07-05 16:25:27,912][INFO ][li.BulkLMI.BulkLMI] ==== Deepen with (0,)
[2023-07-05 16:25:28,036][INFO ][li.BulkLMI.BulkLMI] Epochs: 100, step: 10


None


[2023-07-05 16:25:44,112][INFO ][li.BulkLMI.BulkLMI] Epoch 10 | Loss 0.30305
[2023-07-05 16:25:58,521][INFO ][li.BulkLMI.BulkLMI] Epoch 20 | Loss 0.15311
[2023-07-05 16:26:12,959][INFO ][li.BulkLMI.BulkLMI] Epoch 30 | Loss 0.29747
[2023-07-05 16:26:27,254][INFO ][li.BulkLMI.BulkLMI] Epoch 40 | Loss 0.33624
[2023-07-05 16:26:41,967][INFO ][li.BulkLMI.BulkLMI] Epoch 50 | Loss 0.29719
[2023-07-05 16:26:56,367][INFO ][li.BulkLMI.BulkLMI] Epoch 60 | Loss 0.32929
[2023-07-05 16:27:10,790][INFO ][li.BulkLMI.BulkLMI] Epoch 70 | Loss 0.15786
[2023-07-05 16:27:25,232][INFO ][li.BulkLMI.BulkLMI] Epoch 80 | Loss 0.23273
[2023-07-05 16:27:39,716][INFO ][li.BulkLMI.BulkLMI] Epoch 90 | Loss 0.20142


In [34]:
loaded_clip_data.shape

(100000, 768)

In [31]:
loaded_queries.shape, loaded_queries_seq.shape

((10000, 32), (10000, 768))

In [112]:
anns = []
dists_all = []
k=10
s = time.time()
for i, query in tqdm(enumerate(loaded_queries[:100]), position=0, leave=True):
    pred_leaf_nodes = blmi.search(query, stop_condition=20_000)[0]
    object_ids = []
    for pred_leaf_node in pred_leaf_nodes:
        leaf_node = blmi.nodes.get(pred_leaf_node)
        if leaf_node is not None:
            object_ids.extend(leaf_node.object_ids)

    dists = pairwise_cosine(
        [loaded_queries_seq[i]],
        loaded_clip_data.loc[object_ids]
    )
    nns_idxs = np.argsort(dists)[:, :k][0]
    anns.append(data.loc[object_ids].index[nns_idxs].tolist())
    dists_all.append(dists[0][nns_idxs].tolist())
time.time() - s

100it [01:24,  1.16it/s]


84.02758169174194

In [113]:
np.mean([np.intersect1d(anns, loaded_gt[i][:10]).shape[0] for i in range(100)])

9.11

In [79]:
data.loc[loaded_gt[1][:10]].category_nn

14347    1
82848    1
79302    1
85923    1
6016     1
67067    1
29567    1
54566    1
34591    1
11620    1
Name: category_nn, dtype: int64

In [85]:
blmi.dump_structure()

,type,children
key,,
"(0,)",InnerNode,10
"(0, 0)",LeafNode,18705
"(0, 1)",LeafNode,8489
"(0, 2)",LeafNode,8641
"(0, 3)",LeafNode,3233
"(0, 4)",LeafNode,11842
"(0, 5)",LeafNode,11536
"(0, 6)",LeafNode,8315
"(0, 7)",LeafNode,10881


In [102]:
pred_leaf_nodes = blmi.search(loaded_queries[1], stop_condition_leaf=2)[0]
pred_leaf_nodes

[(0, 1)]

In [99]:
blmi.nodes[(0, )].nn.predict_proba(data_X_to_torch([loaded_queries[1]]))

torch.Size([1, 10])


(array([[3.4948125e-06, 9.9999630e-01, 1.0330035e-08, 4.1298864e-10,
         3.3691144e-10, 1.6628579e-07, 6.0550436e-12, 3.2097949e-10,
         6.6683668e-08, 1.2860399e-08]], dtype=float32),
 array([1, 0, 5, 8, 9, 2, 3, 4, 7, 6]))

#### Debugging finding nothing for query idx=1

In [89]:
np.argsort(res[0])[::-1]

array([[6, 3, 7, 4, 8, 2, 0, 9, 5, 1]])

In [103]:
res = nn.predict_proba(data_X_to_torch([loaded_queries[1]]))
res

(array([[6.0279710e-08, 9.9999917e-01, 5.7332549e-08, 3.1544808e-11,
         2.9970999e-09, 5.3713347e-07, 2.2266798e-11, 5.5535680e-11,
         4.1810239e-08, 1.7528657e-07]], dtype=float32),
 array([[1, 5, 9, 0, 2, 8, 4, 7, 3, 6]]))

In [68]:
i=1
argsorted = np.argsort(res[0][i])[::-1]
argsorted

array([1, 5, 9, 0, 2, 8, 4, 7, 3, 6])

In [ ]:
mean_cats_covered, mean_objects_covered, cats_all, objs_all = get_partitioning_quality_nn(
    nn, loaded_queries, data, loaded_gt, category_nn
)

In [74]:
overall_sum = 0
overall_objects_sum = 0
argsorted = np.argsort(res[0][i])[::-1]
idx = 0
while overall_sum < 9:
    overall_sum += np.sum(data.loc[loaded_gt[i][:10]].category_nn == argsorted[idx])
    overall_objects_sum += np.sum(category_nn == argsorted[idx])
    #overall_objects_sum += np.sum(pred_positions == argsorted[idx])
    idx += 1

In [78]:
data.loc[loaded_gt[i][:10]].category_nn

14347    1
82848    1
79302    1
85923    1
6016     1
67067    1
29567    1
54566    1
34591    1
11620    1
Name: category_nn, dtype: int64

In [75]:
overall_sum, overall_objects_sum

(10, 9226)

In [77]:
idx

1

In [195]:
losses = nn.train_batch(train_loader, epochs=100, logger=LOG)

[2023-07-05 10:39:50,007][INFO ][__main__] Epochs: 100, step: 10
[2023-07-05 10:40:03,091][INFO ][__main__] Epoch 10 | Loss 0.20313
[2023-07-05 10:40:15,178][INFO ][__main__] Epoch 20 | Loss 0.24293
[2023-07-05 10:40:26,946][INFO ][__main__] Epoch 30 | Loss 0.19478
[2023-07-05 10:40:39,214][INFO ][__main__] Epoch 40 | Loss 0.16695
[2023-07-05 10:40:51,460][INFO ][__main__] Epoch 50 | Loss 0.20057
[2023-07-05 10:41:03,348][INFO ][__main__] Epoch 60 | Loss 0.15200
[2023-07-05 10:41:15,394][INFO ][__main__] Epoch 70 | Loss 0.15517
[2023-07-05 10:41:27,504][INFO ][__main__] Epoch 80 | Loss 0.15959
[2023-07-05 10:41:39,342][INFO ][__main__] Epoch 90 | Loss 0.19899


In [196]:
%%time
category_nn = nn.predict(data_X_to_torch(data.drop(['category', 'category_nn'], axis=1, errors='ignore')))
data['category_nn'] = category_nn
mean_cats_covered, mean_objects_covered, cats_all, objs_all = get_partitioning_quality_nn(nn, loaded_queries, data, loaded_gt, category_nn)
mean_cats_covered, mean_objects_covered

100%|██████████| 10000/10000 [00:21<00:00, 457.29it/s]
[2023-07-05 10:42:12,760][INFO ][__main__] mean_cats_covered=2.0164, mean_objects_covered=22370.4726


CPU times: user 20.8 s, sys: 83.5 ms, total: 20.9 s
Wall time: 22 s


(2.0164, 22370.4726)

#### More training makes the NN worse off

In [179]:
data.columns

Index([         0,          1,          2,          3,          4,          5,
                6,          7,          8,          9,         10,         11,
               12,         13,         14,         15,         16,         17,
               18,         19,         20,         21,         22,         23,
               24,         25,         26,         27,         28,         29,
               30,         31, 'category'],
      dtype='object')

In [180]:
category_nn = nn.predict(data_X_to_torch(data.drop('category', axis=1)))

In [182]:
category_nn.shape

(100000,)

In [183]:
data['category_nn'] = category_nn

In [185]:
data.query('category_nn != category').shape

(7576, 34)

In [187]:
%time mean_cats_covered, mean_objects_covered, cats_all, objs_all = get_partitioning_quality_nn(nn, loaded_queries, data, loaded_gt, result_wo)

100%|██████████| 10000/10000 [00:22<00:00, 451.88it/s]
[2023-07-05 10:32:01,976][INFO ][__main__] mean_cats_covered=1.9818, mean_objects_covered=22495.2752


CPU times: user 20.7 s, sys: 96.2 ms, total: 20.8 s
Wall time: 22.2 s


In [189]:
%time mean_cats_covered, mean_objects_covered, cats_all, objs_all = get_partitioning_quality_nn(nn, loaded_queries, data, loaded_gt, category_nn)

100%|██████████| 10000/10000 [00:21<00:00, 465.98it/s]
[2023-07-05 10:32:38,376][INFO ][__main__] mean_cats_covered=1.9752, mean_objects_covered=22207.6475


CPU times: user 20.2 s, sys: 76.2 ms, total: 20.3 s
Wall time: 21.5 s


In [157]:
kmeans.index.search(np.array(queries_prep[:1]).astype(np.float32), 3)

(array([[0.31023687, 0.39056206, 0.39874762]], dtype=float32),
 array([[848, 891, 235]]))

In [156]:
result[loaded_gt[0] - 1]

array([235, 235, 235, 891, 891, 848, 848, 891, 891, 848, 891, 891, 891,
       235, 693, 625, 625, 625, 891, 549, 693, 549, 235,  94, 547, 848,
       891, 891, 963, 625, 779, 693, 235, 963, 891, 625, 549, 891, 779,
       848, 496, 891, 625, 489, 188, 919, 848, 301, 848, 848, 693, 683,
       500, 891, 891,  51, 143, 693, 166, 891, 223, 625, 891, 672, 705,
       223, 656, 891, 625, 625, 625, 625,  51,  51, 166, 698, 625, 848,
       891, 891, 693, 143,  51, 848, 683, 693, 549, 891, 848, 848, 848,
       301, 891, 891, 891,  51, 848, 683, 875, 869])

In [155]:
loaded_gt[0] - 1

array([79171, 15734, 22336,   230, 74172, 41078, 38158, 71848, 69014,
       92810, 99972, 79895, 13235, 64155, 86178, 55217, 60621, 64727,
       86340, 63838, 52856,   884, 83735, 87456, 36441, 24313, 73878,
        5984, 20969, 60558,  5413, 26293, 42585, 41369, 22972,  4414,
       23974, 82380, 15424, 79752, 19744, 74572, 40381,  8734, 70296,
       23883, 92500, 66708, 91231, 61307, 94072,  4326, 25524,  7179,
       12400,  5450, 59682, 17543, 53817, 52101, 85601,   897, 76552,
       18966, 37888, 50122, 80242, 35692, 73073, 31648, 55411, 13492,
       67225, 21471, 81543, 70032, 91178, 97386, 94425, 76537, 64252,
       29032, 87036, 38032, 80912, 70409, 83587, 44951,  6910, 65793,
       45783,  5491, 35204, 21689, 16710, 95027, 19960,  1707, 68449,
       20332], dtype=int32)

In [146]:
loaded_queries[0]

array([-0.06877425, -0.11037602,  0.05646035, -0.11508854, -0.03577028,
        0.09110859,  0.07477964,  0.10778218,  0.04116842,  0.00877987,
       -0.06257925, -0.17031246,  0.00780125, -0.099485  ,  0.04967897,
       -0.12064676,  0.0035742 , -0.07642894,  0.08400699, -0.03188556,
       -0.01020392,  0.07147753, -0.01378688, -0.10539152, -0.00040444,
       -0.00291365,  0.12945361,  0.05624911,  0.00785595,  0.01020052,
        0.06252866, -0.00992952], dtype=float32)

In [11]:
import h5py
import pandas as pd
import pickle
from tqdm import tqdm
from li.utils import pairwise_cosine
import time
import logging
import numpy as np
import os
from li.BaseLMI import cluster_kmeans_faiss, cluster_kmedoids
from li.BaseLMI import BaseLMI
prepare_data_cluster_kmedoids = BaseLMI.prepare_data_cluster_kmedoids
collect_predictions_kmedoids = BaseLMI.collect_predictions_kmedoids
from li.model import NeuralNetwork, data_X_to_torch, data_to_torch
import argparse
from datetime import datetime
from sklearn import preprocessing


In [12]:
k=10
n_categories=1000
epochs=100
lr=0.1
model_type='MLP'

In [ ]:
nn = NeuralNetwork(
    input_dim=data.shape[1], output_dim=n_categories, lr=lr, model_type=model_type
)
data_x, data_y = data_to_torch(data_prep[:100_000], result[:100_000])
#data_x, data_y = data_to_torch(data_part, labels)
LOG.info(f'Starting training')
losses = nn.train(data_x, data_y, epochs=epochs, logger=LOG)

## gauge the nn performance:


In [193]:
%%time
category_nn = nn.predict(data_X_to_torch(data.drop(['category', 'category_nn'], axis=1, errors='ignore')))
data['category_nn'] = category_nn
mean_cats_covered, mean_objects_covered, cats_all, objs_all = get_partitioning_quality_nn(nn, loaded_queries, data, loaded_gt, category_nn)
mean_cats_covered, mean_objects_covered

100%|██████████| 10000/10000 [00:22<00:00, 442.79it/s]
[2023-07-05 10:37:17,411][INFO ][__main__] mean_cats_covered=1.9752, mean_objects_covered=22207.6475


CPU times: user 21.2 s, sys: 104 ms, total: 21.3 s
Wall time: 22.7 s


(1.9752, 22207.6475)

In [ ]:
losses = nn.train(data_x, data_y, epochs=epochs, logger=LOG)

In [ ]:
data_x, data_y = data_to_torch(data_prep[100_000:200_000], result[100_000:200_000])
#data_x, data_y = data_to_torch(data_part, labels)
LOG.info(f'Starting training')
losses = nn.train(data_x, data_y, epochs=epochs, logger=LOG)

In [15]:
np.arange(0, 10_000_000, 100_000)

array([      0,  100000,  200000,  300000,  400000,  500000,  600000,
        700000,  800000,  900000, 1000000, 1100000, 1200000, 1300000,
       1400000, 1500000, 1600000, 1700000, 1800000, 1900000, 2000000,
       2100000, 2200000, 2300000, 2400000, 2500000, 2600000, 2700000,
       2800000, 2900000, 3000000, 3100000, 3200000, 3300000, 3400000,
       3500000, 3600000, 3700000, 3800000, 3900000, 4000000, 4100000,
       4200000, 4300000, 4400000, 4500000, 4600000, 4700000, 4800000,
       4900000, 5000000, 5100000, 5200000, 5300000, 5400000, 5500000,
       5600000, 5700000, 5800000, 5900000, 6000000, 6100000, 6200000,
       6300000, 6400000, 6500000, 6600000, 6700000, 6800000, 6900000,
       7000000, 7100000, 7200000, 7300000, 7400000, 7500000, 7600000,
       7700000, 7800000, 7900000, 8000000, 8100000, 8200000, 8300000,
       8400000, 8500000, 8600000, 8700000, 8800000, 8900000, 9000000,
       9100000, 9200000, 9300000, 9400000, 9500000, 9600000, 9700000,
       9800000, 9900

In [ ]:
from sklearn import preprocessing
from sklearn.cluster import KMeans

kmeans = KMeans().fit(preprocessing.normalize(X))

In [133]:
%%time
clusters = DBSCAN(
    eps=0.2, min_samples=4, metric='cosine', leaf_size=9, p=0.005
).fit(data_s.values)
# eps = 0.5

KeyboardInterrupt: 

In [ ]:
unique_result = np.unique(clusters.labels_, return_counts=True)

In [114]:
unique_result

(array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]),
 array([11172, 88727,     4,     4,     1,     6,     4,     3,    11,
            4,     1,     3,     4,     3,     4,     5,     4,     4,
            4,     3,     3,     2,     4,     4,     2,     2,     3,
            5,     4]))

In [115]:
unique_result[1].max()

88727

In [126]:
from os import listdir

In [127]:
r = listdir('/auto/brno12-cerit/nfs4/home/tslaninakova/sisap-challenge/results-dbscan')

In [128]:
min([int(r_.split('.csv')[0].split('-')[-1]) + int(r_.split('.csv')[0].split('-')[-2]) for r_ in r])

99916

In [129]:
[int(r_.split('.csv')[0].split('-')[-1]) + int(r_.split('.csv')[0].split('-')[-2]) for r_ in r].index(56796)

ValueError: 56796 is not in list

In [125]:
r[12]

'2023-07-04--17-12-10-28398-28398.csv'

In [110]:
unique_result[1][0]

11172

In [111]:
data_s = data.loc[100_000:200_000]

In [112]:
clusters2 = DBSCAN(
    eps=0.2, min_samples=4, metric='cosine', leaf_size=9, p=0.005
).fit(data_s.values)

In [113]:
unique_result2 = np.unique(clusters2.labels_, return_counts=True)
unique_result2[1][0]

11346

In [101]:
data_1 = data.loc[:100_000]
data_1 = data_1.loc[clusters.labels_ == -1]
data_1.shape

(72320, 32)

In [102]:
data_2 = data.loc[100_000:200_000]
data_2 = data_2.loc[clusters2.labels_ == -1]
data_2.shape

(71964, 32)

In [103]:
pd.concat([data_1, data_2])

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
1,0.142266,0.121459,-0.054467,-0.058987,0.049365,-0.107044,0.123232,0.003732,0.044797,-0.118452,...,0.063983,0.037666,-0.089511,0.077113,-0.118540,0.058264,0.065489,0.028578,-0.089683,0.126804
2,0.213747,-0.011409,-0.015184,0.149988,0.106374,0.082410,-0.120636,-0.061065,0.068683,0.033811,...,-0.098325,-0.013612,0.004494,-0.046666,-0.043621,-0.086326,-0.011769,-0.047742,0.020944,-0.030367
3,0.069696,-0.213891,0.132873,-0.150283,-0.049209,0.080035,-0.023633,0.017212,-0.208762,0.053342,...,0.035556,0.023209,-0.048412,-0.051262,0.019094,-0.036998,0.006380,0.011270,-0.029592,0.076340
4,0.136413,-0.124811,-0.154160,0.239878,0.002287,0.145432,-0.069391,-0.006645,0.124315,0.054875,...,-0.045162,0.027401,0.072960,-0.035548,-0.003664,-0.007761,-0.065726,-0.013585,-0.012794,-0.030778
5,-0.389241,0.076076,-0.072001,0.103547,-0.058551,0.033468,0.017906,0.094761,0.034065,-0.220189,...,-0.051880,-0.016713,0.018730,0.050222,-0.021090,0.004709,0.043465,-0.049461,-0.052292,0.027773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,-0.016623,0.261374,-0.108525,-0.135211,0.032195,0.070787,0.079449,-0.092412,-0.100304,0.067358,...,0.081138,-0.013030,0.065910,-0.052886,0.022053,0.036298,-0.059659,0.027475,-0.006544,-0.060666
199996,-0.135043,0.058869,-0.033821,-0.066608,-0.032245,0.102940,0.085796,0.028704,0.096673,-0.053070,...,-0.040772,-0.024435,0.013444,0.064723,0.059284,-0.020601,0.065155,-0.004740,-0.011218,-0.096802
199997,0.056166,0.045453,-0.165639,-0.001400,0.177127,-0.021528,-0.009026,-0.040884,0.220318,-0.039731,...,-0.096172,0.015361,-0.034067,-0.060641,0.030858,0.005659,0.010588,0.035196,-0.021983,0.015925
199998,0.172810,0.136016,0.052640,-0.048010,0.118766,-0.002332,0.111289,-0.026583,-0.051066,-0.125786,...,0.088246,-0.000755,0.012355,0.057855,-0.032556,0.118518,-0.025377,0.024111,0.063948,-0.016831


In [104]:
clusters3 = DBSCAN(
    eps=0.1, min_samples=5, metric='cosine', leaf_size=10, p=0.01
).fit(pd.concat([data_1, data_2]).values)

In [105]:
unique_result3 = np.unique(clusters3.labels_, return_counts=True)
unique_result3[1][0]

140027

In [87]:
data_s.loc[clusters.labels_ == -1].shape

(72315, 32)

In [ ]:
data.

In [88]:
clusters_2 = DBSCAN(
    eps=0.1, min_samples=5, metric='cosine', leaf_size=10, p=0.01
).fit(data_s.loc[clusters.labels_ == -1])

In [89]:
unique_result_2 = np.unique(clusters_2.labels_, return_counts=True)

In [90]:
unique_result_2[1][0]

72315

In [75]:
unique_result[1][0]

78487

In [ ]:
%time clusters = DBSCAN(eps=3, min_samples=2).fit(data_s)

In [53]:
from scipy.spatial.distance import cdist

In [ ]:
loaded_clip_data[:1_000_000]

In [58]:
loaded_queries[0].shape

(32,)

In [ ]:
loaded_queries[0]

In [61]:
from scipy import sparse

In [ ]:
data_part_sparse = sparse.csr_matrix(clip_data.loc[object_ids])
query_part_sparse = sparse.csr_matrix(loaded_queries_seq[i])

In [59]:
%%time
res = cdist(
    [loaded_queries_seq[0]], loaded_clip_data[:1_000_000], metric='cosine'
)

CPU times: user 4.19 s, sys: 2.2 s, total: 6.39 s
Wall time: 6.68 s


In [64]:
%%time
res = cdist(
    sparse.csr_matrix(loaded_queries_seq[:2]), sparse.csr_matrix(loaded_clip_data[:1_000_000]), metric='cosine'
)

ValueError: XA must be a 2-dimensional array.